# 快速构建基于Milvus的文本-图像搜索引擎

## 准备
确保系统有GPU，并且python版本为3.10，当前不支持python3.12
### 下载依赖

In [8]:
! python -m pip install -q towhee gradio opencv-python

### 准备数据
数据集包含100个图像类别，每个类别中包含10张图片。数据集可通过Github下载： [Github](https://github.com/towhee-io/examples/releases/download/data/reverse_image_search.zip). 

数据集包含如下三个部分：
- **train**: 候选图片目录;
- **test**: 测试图片目录;
- **reverse_image_search.csv**: csv文件，每张图片包含： ***id***, ***path***,  ***label*** ;


In [ ]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/reverse_image_search.zip -O
! unzip -q -o reverse_image_search.zip

In [ ]:
import pandas as pd

df = pd.read_csv('reverse_image_search.csv')
df.head()

下面的fuction是作为text-image search的辅助
- **read_images(results)**: 通过图片ID读入图片，返回图片列表;

In [10]:
import cv2
from towhee.types.image import Image

id_img = df.set_index('id')['path'].to_dict()
def read_images(results):
    imgs = []
    for re in results:
        path = id_img[re.id]
        imgs.append(Image(cv2.imread(path), 'BGR'))
    return imgs


### 创建Milvus链接

为了防止版本冲突情况，确保grpcio的版本限制在如下的范围内，下面还引入了Milvus，是因为源码中没有启动Milvus，所以需要手动安装milvus然后启动milvus服务

In [ ]:
! pip install "grpcio>=1.49.1,<=1.53.0" pymilvus milvus

如果你已经安装了pymilvus导致了版本冲突问题，请运行如下代码，重新安装pymilvus

In [ ]:
! pip uninstall pymilvus -y

现在创建一个 `text_image_search` 的milvus collection，使用 [L2 distance metric](https://milvus.io/docs/metric.md#Euclidean-distance-L2) 和 [IVF_FLAT index](https://milvus.io/docs/index.md#IVF_FLAT)索引.

In [ ]:
from milvus import default_server  
default_server.start()  

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

def create_milvus_collection(collection_name, dim):
    connections.connect("default",host='localhost', port='19530')
    
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.INT64, descrition='ids', is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='text image search')
    collection = Collection(name=collection_name, schema=schema)

    # 为集合创建 IVF_FLAT 索引.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":512}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

collection = create_milvus_collection('text_image_search', 512)

## Text Image Search

使用 [Towhee](https://towhee.io/), 建立一个文本图像搜索引擎。

<img src="./workflow.png" width = "60%" height = "60%" align=center />

### 使用CLIP模型对文本和图片进行向量化


使用 [CLIP](https://openai.com/blog/clip/) 提取图像或文本的特征，该模型能够通过联合训练图像编码器和文本编码器来最大化余弦相似度，从而生成文本和图像的嵌入表示。

In [ ]:
from towhee import ops, pipe, DataCollection
import numpy as np

### 从魔搭社区下载模型
下面的两段代码是从魔搭社区下载模型，建议自己手动下载clip-vit-base-patch16，放到model文件夹下

In [ ]:
! pip install modelscope
! modelscope download --model openai-mirror/clip-vit-base-patch16 --local_dir /model

In [ ]:
p = (
    pipe.input('path')
    .map('path', 'img', ops.image_decode.cv2('rgb'))
    .map('img', 'vec', ops.image_text_embedding.clip(model_name='model', modality='image'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .output('img', 'vec')
)
# DataCollection(p('./teddy.png')).show()

In [ ]:
p2 = (
    pipe.input('text')
    .map('text', 'vec', ops.image_text_embedding.clip(model_name='model', modality='text'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .output('text', 'vec')
)

# DataCollection(p2("A teddybear on a skateboard in Times Square.")).show()

下面是代码释意:

- `map('path', 'img', ops.image_decode.cv2_rgb('rgb'))`: 对于数据的每一行, 读取并且decode `path`下的数据然后放到 `img`中;

- `map('img', 'vec', ops.image_text_embedding.clip(model_name='model', modality='image'/'text'))`：使用 `ops.image_text_embedding.clip` 提取图像或文本的嵌入特征，该操作符来自 [Towhee hub](https://towhee.io/image-text-embedding/clip)。此操作符支持多种模型，包括 `clip_vit_base_patch16`、`clip_vit_base_patch32`、`clip_vit_large_patch14`、`clip_vit_large_patch14_336` 等。

### 将图片向量数据导入Milvus中

我们首先将已经由 `clip_vit_base_patch16` 模型处理好的图片向量化数据插入Milvus中用于后面的检索。 Towhee 提供了[method-chaining style API](https://towhee.readthedocs.io/en/main/index.html) 因此，用户可以使用这些操作符组装一个数据处理管道。这意味着用户可以根据自己的需求，将不同的操作符（如图像和文本嵌入提取操作符）组合起来，创建复杂的数据处理流程，以实现特定的功能或任务。例如，在图像检索、文本匹配或其他涉及多模态数据处理的应用场景中，通过这种方式可以灵活地构建解决方案。

In [ ]:
collection = create_milvus_collection('text_image_search', 512)

def read_csv(csv_path, encoding='utf-8-sig'):
    import csv
    with open(csv_path, 'r', encoding=encoding) as f:
        data = csv.DictReader(f)
        for line in data:
            yield int(line['id']), line['path']

p3 = (
    pipe.input('csv_file')
    .flat_map('csv_file', ('id', 'path'), read_csv)
    .map('path', 'img', ops.image_decode.cv2('rgb'))
    .map('img', 'vec', ops.image_text_embedding.clip(model_name='model', modality='image', device=0))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .map(('id', 'vec'), (), ops.ann_insert.milvus_client(host='127.0.0.1', port='19530', collection_name='text_image_search'))
    .output()
)

ret = p3('reverse_image_search.csv')


In [11]:
collection.load()

In [12]:
print('Total number of inserted data is {}.'.format(collection.num_entities))

Total number of inserted data is 0.


### 开始向量化检索

现在，候选图像的嵌入向量已经插入到 Milvus 中，我们可以对其进行最近邻查询。同样，我们使用 Towhee 来加载输入文本、计算嵌入向量，并将该向量作为 Milvus 的查询条件。由于 Milvus 仅返回图像 ID 和距离值，我们提供了一个 `read_images` 函数，根据 ID 获取原始图像并进行展示。

In [ ]:
import pandas as pd
import cv2

def read_image(image_ids):
    df = pd.read_csv('reverse_image_search.csv')
    id_img = df.set_index('id')['path'].to_dict()
    imgs = []
    decode = ops.image_decode.cv2('rgb')
    for image_id in image_ids:
        path = id_img[image_id]
        imgs.append(decode(path))
    return imgs


p4 = (
    pipe.input('text')
    .map('text', 'vec', ops.image_text_embedding.clip(model_name='model', modality='text'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .map('vec', 'result', ops.ann_search.milvus_client(host='127.0.0.1', port='19530', collection_name='text_image_search', limit=5))
    .map('result', 'image_ids', lambda x: [item[0] for item in x])
    .map('image_ids', 'images', read_image)
    .output('text', 'images')
)

DataCollection(p4("A white dog")).show()
DataCollection(p4("A black dog")).show()

## 使用Gradio构建一个应用

In [ ]:
search_pipeline = (
    pipe.input('text')
    .map('text', 'vec', ops.image_text_embedding.clip(model_name='model', modality='text'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .map('vec', 'result', ops.ann_search.milvus_client(host='127.0.0.1', port='19530', collection_name='text_image_search', limit=5))
    .map('result', 'image_ids', lambda x: [item[0] for item in x])
    .output('image_ids')
)

def search(text):
    df = pd.read_csv('reverse_image_search.csv')
    id_img = df.set_index('id')['path'].to_dict()
    imgs = []
    image_ids = search_pipeline(text).to_list()[0][0]
    return [id_img[image_id] for image_id in image_ids]


在高版本的gradio中，已经不支持gradio.inputs.xxx和gradio.outputs.xxx，可直接使用gradio.TextBox或者gradio.Image
你可以使用如下代码更新一下你的gradio

In [ ]:
! pip install --upgrade gradio

In [ ]:
import gradio

interface = gradio.Interface(search, 
                             gradio.Textbox(lines=1),
                             [gradio.Image(type="filepath", label=None) for _ in range(5)]
                            )

interface.launch(inline=True, share=True)